In [1]:
import pandas as pd
import yfinance as yf
import funzioni as fx
import plotly.offline as pyo
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.models import load_model
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os

pd.set_option("display.max_columns", None)

def visualizza_grafico(ticker):
    import plotly.graph_objs as go
    fig = fx.grafico_base(ticker)
    tg = go.Scatter(
        x = ticker[ticker['Target'] == True].index,
        y = ticker[ticker['Target'] == True]['Close'],
        mode = 'markers',
        marker = dict(
            size = 15,
            color = 'rgba(0, 250, 0, .9)'
        ),
        name = 'target'
    )
    fig.add_trace(tg, row=1, col=1)
    pyo.plot(fig, filename="prova.html", auto_open=True)

def addestramento(model, features, target):    
    X, Y = fx.converti_in_XY(features, target)
    Y = np.where(Y, 1, 0)
    class_weights = compute_class_weight('balanced', classes=np.unique(Y), y=Y)
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(class_weights) 

    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    if model == 0:
        model = keras.Sequential([
            layers.LSTM(200, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
            layers.Dropout(0.2),
            layers.LSTM(200, return_sequences=True),  # Nuovo layer LSTM 1
            layers.Dropout(0.2),
            layers.LSTM(200, return_sequences=True),  # Nuovo layer LSTM 2
            layers.Dropout(0.2),
            layers.LSTM(200),  # Ultimo layer LSTM
            layers.Dropout(0.2),
            layers.Dense(1, activation='sigmoid')
        ])
    custom_optimizer = Adam(learning_rate=0.0001)

    model.compile(
        optimizer=custom_optimizer,
        loss='binary_crossentropy',
        metrics=[
            Precision(name='precision'),
            Recall(name='recall'),
            AUC(name='auc'),
            'accuracy'
        ]
    )

    scaler = StandardScaler()
    n_samples, n_timesteps, n_features = X.shape
    X_reshape = X.reshape((n_samples, n_timesteps * n_features))
    X_scaled = scaler.fit_transform(X_reshape)
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)
    X_train_scaled = X_train.reshape((-1, n_timesteps, n_features))
    X_test_scaled = X_test.reshape((-1, n_timesteps, n_features))
    
    model.fit(
        X_train_scaled, Y_train,
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        validation_data=(X_test_scaled, Y_test),
        callbacks=[
            early_stopping
        ]
    )

    loss, precision, recall, auc, accuracy = model.evaluate(X_test_scaled, Y_test)
    print(f"\033[42mLoss: {loss}, Precision: {precision}, Recall: {recall}, AUC: {auc}, Accuracy: {accuracy}\033[0m")
    with open("result.txt", "a") as f:
        f.write(f"Loss: {loss}, Precision: {precision}, Recall: {recall}, AUC: {auc}, Accuracy: {accuracy}\n")
    return model, scaler



In [2]:
elenco_usin = pd.read_excel("DeGiro con formule.xlsx")
elenco_usin = elenco_usin.loc[elenco_usin["Categoria"] != "D", ["USIN", "Ticker"]]
elenco_usin = elenco_usin.sample(frac=1)
tot_usin = len(elenco_usin)
i = 1
file_modello = "C:/Users/932197/Documents/dati_locali/LSTM.keras"
if os.path.exists(file_modello):
    model = load_model(file_modello)
else:
    model = 0
for index, usin in elenco_usin.iterrows():       
    print(f"\r{i} di {tot_usin}) Caricamento ticker {usin['Ticker']}                           ", end=' ', flush=True)
    ticker = yf.download(usin["USIN"], start='2005-01-01', end='2023-07-31', progress=False)
    if len(ticker) < 500:
        continue
    else:
        ticker = fx.crea_indicatori(ticker)
        features, target = fx.features_e_target(ticker)
        ticker["Target"] = target
        model, scaler = addestramento(model, features=features, target=target) 
        model.save(file_modello)
        i += 1

print("fine addestramento")

1 di 2581) Caricamento ticker NVR                            [ 0.51897577 13.6746988 ]
Epoch 1/100
114/114 [==============================] - 34s 163ms/step - loss: 0.6093 - precision: 0.0673 - recall: 0.7231 - auc: 0.7355 - accuracy: 0.6313 - val_loss: 0.5463 - val_precision: 0.1369 - val_recall: 0.9167 - val_auc: 0.8834 - val_accuracy: 0.7676
Epoch 2/100
114/114 [==============================] - 16s 140ms/step - loss: 0.4694 - precision: 0.1310 - recall: 0.8769 - auc: 0.8562 - accuracy: 0.7874 - val_loss: 0.4425 - val_precision: 0.1698 - val_recall: 1.0000 - val_auc: 0.9039 - val_accuracy: 0.8062
Epoch 3/100
114/114 [==============================] - 15s 128ms/step - loss: 0.3914 - precision: 0.1364 - recall: 0.8769 - auc: 0.8924 - accuracy: 0.7968 - val_loss: 0.4352 - val_precision: 0.1522 - val_recall: 0.9722 - val_auc: 0.9142 - val_accuracy: 0.7841
Epoch 4/100
114/114 [==============================] - 14s 125ms/step - loss: 0.3543 - precision: 0.1392 - recall: 0.9231 - auc: 0.90

Epoch 4/100
114/114 [==============================] - 14s 123ms/step - loss: 0.3533 - precision: 0.1035 - recall: 0.8621 - auc: 0.9360 - accuracy: 0.8786 - val_loss: 0.3638 - val_precision: 0.0930 - val_recall: 0.9231 - val_auc: 0.9073 - val_accuracy: 0.8700
Epoch 5/100
114/114 [==============================] - 15s 132ms/step - loss: 0.2789 - precision: 0.1220 - recall: 0.8793 - auc: 0.9611 - accuracy: 0.8970 - val_loss: 0.3288 - val_precision: 0.0909 - val_recall: 0.9231 - val_auc: 0.9000 - val_accuracy: 0.8667
Epoch 6/100
114/114 [==============================] - 15s 133ms/step - loss: 0.2096 - precision: 0.1276 - recall: 0.9655 - auc: 0.9748 - accuracy: 0.8940 - val_loss: 0.3110 - val_precision: 0.0803 - val_recall: 0.8462 - val_auc: 0.9010 - val_accuracy: 0.8590
Epoch 7/100
114/114 [==============================] - 17s 146ms/step - loss: 0.1820 - precision: 0.1399 - recall: 0.9483 - auc: 0.9784 - accuracy: 0.9061 - val_loss: 0.2718 - val_precision: 0.0938 - val_recall: 0.9231 -

10/10 [==============================] - 1s 125ms/step - loss: 0.3004 - precision: 0.3457 - recall: 0.9655 - auc: 0.9481 - accuracy: 0.8194 - val_loss: 0.5950 - val_precision: 0.2174 - val_recall: 0.8333 - val_auc: 0.8357 - val_accuracy: 0.7467
Epoch 6/100
10/10 [==============================] - 1s 124ms/step - loss: 0.2768 - precision: 0.4058 - recall: 0.9655 - auc: 0.9582 - accuracy: 0.8595 - val_loss: 0.4427 - val_precision: 0.2778 - val_recall: 0.8333 - val_auc: 0.8623 - val_accuracy: 0.8133
Epoch 7/100
10/10 [==============================] - 1s 132ms/step - loss: 0.2344 - precision: 0.4746 - recall: 0.9655 - auc: 0.9695 - accuracy: 0.8930 - val_loss: 0.3859 - val_precision: 0.2667 - val_recall: 0.6667 - val_auc: 0.8671 - val_accuracy: 0.8267
Epoch 8/100
10/10 [==============================] - 1s 129ms/step - loss: 0.2053 - precision: 0.5600 - recall: 0.9655 - auc: 0.9699 - accuracy: 0.9231 - val_loss: 0.3559 - val_precision: 0.3077 - val_recall: 0.6667 - val_auc: 0.8708 - val_a

Epoch 37/100
10/10 [==============================] - 1s 131ms/step - loss: 0.0782 - precision: 0.7436 - recall: 1.0000 - auc: 0.9946 - accuracy: 0.9666 - val_loss: 0.2926 - val_precision: 0.4000 - val_recall: 0.6667 - val_auc: 0.9312 - val_accuracy: 0.8933
Epoch 38/100
3/3 [==============================] - 0s 47ms/step - loss: 0.3503 - precision: 0.4000 - recall: 0.6667 - auc: 0.9155 - accuracy: 0.8933
Loss: 0.3503151834011078, Precision: 0.4000000059604645, Recall: 0.6666666865348816, AUC: 0.9154589176177979, Accuracy: 0.8933333158493042
4 di 2581) Caricamento ticker RTX                            [ 0.50287993 87.30769231]
Epoch 1/100
114/114 [==============================] - 33s 161ms/step - loss: 0.7215 - precision: 0.0265 - recall: 0.4737 - auc: 0.8778 - accuracy: 0.9061 - val_loss: 0.1866 - val_precision: 0.0933 - val_recall: 1.0000 - val_auc: 0.9770 - val_accuracy: 0.9251
Epoch 2/100
114/114 [==============================] - 15s 135ms/step - loss: 0.2298 - precision: 0.0464 -

114/114 [==============================] - 14s 124ms/step - loss: 0.0264 - precision: 0.3115 - recall: 1.0000 - auc: 0.9978 - accuracy: 0.9884 - val_loss: 0.0535 - val_precision: 0.3889 - val_recall: 1.0000 - val_auc: 0.9958 - val_accuracy: 0.9879
Epoch 30/100
114/114 [==============================] - 14s 124ms/step - loss: 0.0257 - precision: 0.3333 - recall: 1.0000 - auc: 0.9973 - accuracy: 0.9895 - val_loss: 0.0583 - val_precision: 0.3684 - val_recall: 1.0000 - val_auc: 0.9961 - val_accuracy: 0.9868
Epoch 31/100
114/114 [==============================] - 14s 125ms/step - loss: 0.0280 - precision: 0.3167 - recall: 1.0000 - auc: 0.9973 - accuracy: 0.9887 - val_loss: 0.0811 - val_precision: 0.2593 - val_recall: 1.0000 - val_auc: 0.9957 - val_accuracy: 0.9780
Epoch 32/100
114/114 [==============================] - 14s 125ms/step - loss: 0.0301 - precision: 0.3115 - recall: 1.0000 - auc: 0.9971 - accuracy: 0.9884 - val_loss: 0.1064 - val_precision: 0.2000 - val_recall: 1.0000 - val_auc:

...

114/114 [==============================] - 16s 139ms/step - loss: 0.1005 - precision: 0.3006 - recall: 0.9608 - auc: 0.9905 - accuracy: 0.9681 - val_loss: 0.2274 - val_precision: 0.1923 - val_recall: 1.0000 - val_auc: 0.9896 - val_accuracy: 0.9306
Epoch 21/100
114/114 [==============================] - 16s 137ms/step - loss: 0.0846 - precision: 0.2308 - recall: 1.0000 - auc: 0.9904 - accuracy: 0.9532 - val_loss: 0.1173 - val_precision: 0.3125 - val_recall: 1.0000 - val_auc: 0.9894 - val_accuracy: 0.9637
Epoch 22/100
114/114 [==============================] - 16s 137ms/step - loss: 0.1098 - precision: 0.2591 - recall: 0.9804 - auc: 0.9899 - accuracy: 0.9604 - val_loss: 0.1922 - val_precision: 0.1892 - val_recall: 0.9333 - val_auc: 0.9887 - val_accuracy: 0.9328
Epoch 23/100
114/114 [==============================] - 16s 137ms/step - loss: 0.0939 - precision: 0.2193 - recall: 0.9804 - auc: 0.9908 - accuracy: 0.9507 - val_loss: 0.1043 - val_precision: 0.3125 - val_recall: 1.0000 - val_auc:

9/9 [==============================] - 0s 49ms/step - loss: 0.4637 - precision: 0.4754 - recall: 0.8529 - auc: 0.8672 - accuracy: 0.8679
Loss: 0.4637220799922943, Precision: 0.4754098355770111, Recall: 0.8529411554336548, AUC: 0.8671687841415405, Accuracy: 0.8678571581840515
14 di 2581) Caricamento ticker GFI                            [0.56693307 4.23507463]
Epoch 1/100
114/114 [==============================] - 38s 201ms/step - loss: 0.9464 - precision: 0.1893 - recall: 0.5648 - auc: 0.6565 - accuracy: 0.6605 - val_loss: 0.5699 - val_precision: 0.2076 - val_recall: 0.5769 - val_auc: 0.7529 - val_accuracy: 0.6993
Epoch 2/100
114/114 [==============================] - 17s 148ms/step - loss: 0.5327 - precision: 0.2683 - recall: 0.7477 - auc: 0.8124 - accuracy: 0.7274 - val_loss: 0.4542 - val_precision: 0.3062 - val_recall: 0.7596 - val_auc: 0.8502 - val_accuracy: 0.7753
Epoch 3/100
114/114 [==============================] - 17s 152ms/step - loss: 0.4542 - precision: 0.3147 - recall: 0.8

114/114 [==============================] - 16s 141ms/step - loss: 0.1109 - precision: 0.7090 - recall: 0.9815 - auc: 0.9888 - accuracy: 0.9499 - val_loss: 0.2083 - val_precision: 0.6144 - val_recall: 0.9038 - val_auc: 0.9690 - val_accuracy: 0.9240
Epoch 31/100
114/114 [==============================] - 16s 143ms/step - loss: 0.1139 - precision: 0.6995 - recall: 0.9861 - auc: 0.9878 - accuracy: 0.9480 - val_loss: 0.2262 - val_precision: 0.5987 - val_recall: 0.8750 - val_auc: 0.9624 - val_accuracy: 0.9185
Epoch 32/100
114/114 [==============================] - 16s 142ms/step - loss: 0.1245 - precision: 0.6807 - recall: 0.9722 - auc: 0.9859 - accuracy: 0.9425 - val_loss: 0.2212 - val_precision: 0.6267 - val_recall: 0.9038 - val_auc: 0.9637 - val_accuracy: 0.9273
Epoch 33/100
114/114 [==============================] - 16s 143ms/step - loss: 0.1512 - precision: 0.6406 - recall: 0.9653 - auc: 0.9836 - accuracy: 0.9314 - val_loss: 0.2032 - val_precision: 0.6013 - val_recall: 0.8846 - val_auc:

16 di 2581) Caricamento ticker RLGT                            [0.57843972 3.68716094]
Epoch 1/100
102/102 [==============================] - 39s 198ms/step - loss: 0.8256 - precision: 0.2517 - recall: 0.6803 - auc: 0.7111 - accuracy: 0.6833 - val_loss: 0.5470 - val_precision: 0.3220 - val_recall: 0.8482 - val_auc: 0.8605 - val_accuracy: 0.7341
Epoch 2/100
102/102 [==============================] - 15s 148ms/step - loss: 0.4530 - precision: 0.3451 - recall: 0.8231 - auc: 0.8669 - accuracy: 0.7649 - val_loss: 0.4176 - val_precision: 0.3984 - val_recall: 0.9107 - val_auc: 0.9141 - val_accuracy: 0.7990
Epoch 3/100
102/102 [==============================] - 15s 151ms/step - loss: 0.3761 - precision: 0.4017 - recall: 0.8707 - auc: 0.9096 - accuracy: 0.8072 - val_loss: 0.3471 - val_precision: 0.4348 - val_recall: 0.8929 - val_auc: 0.9353 - val_accuracy: 0.8260
Epoch 4/100
102/102 [==============================] - 16s 162ms/step - loss: 0.3252 - precision: 0.4561 - recall: 0.8957 - auc: 0.93


KeyboardInterrupt

